<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/magistral_gpt4_aai_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## MISTRAL AI

try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env --quiet
    import colab_env

try:
    import mistralai
    # Re-import after upgrade to ensure new version is loaded in current session if needed
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai --upgrade'.")


api_key = os.environ["MISTRAL_API_KEY"]
mistral_client = Mistral(api_key=api_key)


print("Creating AI agents...")

try:
    # Using mistral_raw_client which is initialized in H4MplWTd2x0K
    agent0 = mistral_client.beta.agents.create(
        model="magistral-medium-latest", # Reverted to the original model name
        description="Agent used to answer something related requests",
        name="agent0",
        # Add tools here if needed, e.g., tools=[{"type": "function", "function": ...}]
    )
    print(f"Mistral Agent (agent0) created with ID: {agent0.id}")

except NameError:
    print("Error: mistral_raw_client is not defined. Ensure the cell initializing it has been run.")
    # Handle the error or exit if mistral_raw_client is strictly required here.
except Exception as e:
    print(f"An error occurred while creating the Mistral Agent: {e}")


# ## OPENAI
from agents import Agent, Runner, handoff

flight_search_agent = Agent(
     name="FlightSearchAgent",
     instructions=(
         "You are a specialized AI assistant for finding flight information. "
        "Your goal is to extract all necessary details (origin, destination, departure date, "
         "return date, preferred airlines, maximum price, minimum stops) from the user's request "
         "and use the 'simulated_flight_search' tool to find flights. "
         "When using the tool, ensure you extract all available parameters from the user's request, "
         "such as 'min_stops' or 'max_price'. "
         "Once you receive the list of flight options from the tool, carefully analyze them. "
         "Then, provide a concise and helpful summary of the BEST 2-3 options to the user, highlighting "
         "key details like airline, exact price, number of stops, and total duration. "
         "Prioritize direct flights and shorter durations if prices are comparable. "
         "If no flights are found, inform the user clearly and suggest adjusting criteria (e.g., dates, price). "
         "If you don't have enough information to search, ask clarifying questions, "
         "e.g., 'What are your desired departure and return dates for your flight from [Origin] to [Destination]?'"
     ),
     model="gpt-4o",
     tools=[simulated_flight_search],
)

## AGENT

In [ ]:
!pip install langchain-openai -q
!pip install colab-env -q
!pip install mistralai -q
!pip install openai-agents -q

In [1]:
# flight_planning_multi_agent_solution_v33.py

import os
import asyncio
import nest_asyncio
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any
from datetime import datetime
import json # Import json for parsing tool arguments

# --- Apply nest_asyncio to allow asyncio.run in environments like Colab ---
nest_asyncio.apply()
print("nest_asyncio applied to allow running asyncio.run in this environment.")

# Ensure colab_env is installed if running in Colab
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env --quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    if "No module named 'mistralai'" in str(e):
        print("Installing mistralai...")
        !pip install mistralai --quiet
        from mistralai import Mistral # Try importing again after install
    else:
        print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai --upgrade'.")
        exit() # Exit if installation or re-import fails


# --- OpenAI Imports (for Assistants API) ---
try:
    from openai import OpenAI as OpenAIClient
except ImportError as e:
    print(f"Error importing openai: {e}")
    if "No module named 'openai'" in str(e):
        print("Installing openai...")
        !pip install openai --quiet
        from openai import OpenAI as OpenAIClient # Try importing again after install
    else:
        print("Please install it: pip install openai --quiet")
        exit() # Exit if installation or re-import fails


# Restoring imports from the 'agents' library as requested by the user
# We are going back to defining agents using agents.Agent so Runner/Handoff work
from agents import Agent, Runner, handoff
from agents import function_tool as tool # Keep this to use the decorator for tool functions
from agents import enable_verbose_stdout_logging

# Enable verbose logging for your agents framework
enable_verbose_stdout_logging()

# --- Set up Environment Variables ---
mistral_api_key = os.environ.get("MISTRAL_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")

if not mistral_api_key:
    raise ValueError("MISTRAL_API_KEY environment variable not set. Please set it.")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it.")

print("API keys loaded successfully.")
print("Environment variables set.")
print('\n')

# --- Initialize External LLM Clients ---
# Keeping these initialized in case the agents.Agent class uses them internally
mistral_raw_client = Mistral(api_key=mistral_api_key)
openai_raw_client = OpenAIClient(api_key=openai_api_key)


# --- Define Tools using your `function_tool as tool` ---

# Pydantic models for internal documentation/clarity and for generating API schemas
class WeatherInput(BaseModel):
    location: str = Field(description="The geographical location, e.g., 'Paris, France'")
    time: str = Field(description="The specific time or date for the forecast, e.g., 'tomorrow 10 AM local time'")

class NotamInput(BaseModel):
    airspace: str = Field(description="The identifier for the airspace, e.g., 'LAX' or 'EHAM'")

class ComplianceInput(BaseModel):
    route_details: str = Field(description="Detailed description of the proposed flight route.")
    regulations: List[str] = Field(description="List of specific aviation regulations to check against.")

class FuelCalcInput(BaseModel):
    aircraft_type: str = Field(description="The type of aircraft, e.g., 'Boeing 747'")
    distance_km: float = Field(description="The flight distance in kilometers.")
    weather_conditions: str = Field(description="Brief description of weather conditions influencing fuel burn.")


@tool
def get_weather_forecast(location: str, time: str) -> str:
    """Fetches detailed weather forecast for a given location and time."""
    print(f"[Tool:get_weather_forecast] Fetching weather for {location} at {time}...")
    if "storm" in location.lower() or "bad weather" in time.lower():
        return "Severe thunderstorms with high winds and low visibility expected. Instrument Flight Rules (IFR) likely."
    return "Clear skies, light winds, good visibility. Visual Flight Rules (VFR) conditions."

@tool
def get_notams(airspace: str) -> str:
    """Retrieves Notices to Airmen (NOTAMs) for a specified airspace."""
    print(f"[Tool:get_notams] Retrieving NOTAMs for {airspace}...")
    if "LAX" in airspace.upper():
        return "NOTAM: LAX Runway 25L closed for maintenance until 2025-07-15. Expect delays."
    if "LONDON HEATHROW" in airspace.upper() or "EGLL" in airspace.upper():
        return "NOTAM: EGLL ATC system experiencing minor disruptions, expect minor flow control."
    return "No significant NOTAMs for this airspace at this time."

@tool
def check_regulatory_compliance(route_details: str, regulations: List[str]) -> str:
    """
    Checks a proposed route against aviation regulations and provides detailed reasoning.
    The underlying Magistral agent will use its reasoning capabilities for this.
    """
    print(f"[Tool:check_regulatory_compliance] Checking regulatory compliance for route: '{route_details}' against: {regulations}")
    compliance_issues = []
    reasoning_steps = []

    if "restricted airspace" in route_details.lower() and not "special clearance" in route_details.lower():
        compliance_issues.append("Violation: Route enters restricted airspace without explicit clearance.")
        reasoning_steps.append("Identified 'restricted airspace' in route details. No mention of 'special clearance'. Regulations require clearance for such entry.")

    # Using the get_weather_forecast tool to dynamically influence compliance check
    # Note: Calling a tool function directly here bypasses the agent's tool-use mechanism.
    current_weather_at_dest = get_weather_forecast("destination", "now") # Hypothetical call
    if "ICAO Annex 2" in regulations and "VFR" in route_details and "IFR" in current_weather_at_dest.lower():
        compliance_issues.append("Violation: Attempting Visual Flight Rules (VFR) in anticipated Instrument Flight Rules (IFR) conditions.")
        reasoning_steps.append("Checked ICAO Annex 2. VFR flights require specific weather minima. Destination weather indicates IFR conditions, making VFR non-compliant.")

    if not compliance_issues:
        return f"Compliance Check: PASSED. All specified regulations ({', '.join(regulations)}) appear to be met. Reasoning: Route avoids restricted zones, adheres to flight rules based on forecast weather, and no critical NOTAMs apply."
    else:
        return f"Compliance Check: FAILED.\nIssues: {'; '.join(compliance_issues)}\nDetailed Reasoning:\n- " + "\n- ".join(reasoning_steps) + "\nRecommended Action: Reroute or seek necessary clearances."

@tool
def calculate_fuel_burn(aircraft_type: str, distance_km: float, weather_conditions: str) -> str:
    """Calculates estimated fuel burn for a given aircraft type, distance and weather."""
    print(f"[Tool:calculate_fuel_burn] Calculating fuel burn for {aircraft_type} over {distance_km:.0f}km with '{weather_conditions}'...")
    base_fuel_rate_per_km = 0.05
    if "Boeing 787" in aircraft_type:
        base_fuel_rate_per_km = 0.08
    fuel_burn = distance_km * base_fuel_rate_per_km
    if "high winds" in weather_conditions.lower() or "thunderstorms" in weather_conditions.lower():
        fuel_burn *= 1.15
        return f"Estimated fuel burn: {fuel_burn:.2f} liters (adjusted for adverse weather)."
    return f"Estimated fuel burn: {fuel_burn:.2f} liters."


# --- Define Agents using your framework's Agent class ---

# Mistral Agent (Magistral for Regulatory & Risk) - Using agents.Agent
# Defining the agent using the agents.Agent class to ensure compatibility with Runner/Handoff
try:
    regulatory_risk_agent = Agent(
        name="RegulatoryRiskAgent",
        instructions=(
            "You are an expert in aviation regulations and risk. "
            "You will receive a proposed flight route and evaluate its compliance and risks. "
            "You are backed by a powerful Mistral AI agent that provides transparent, step-by-step reasoning. "
            "Use your tools (get_notams, check_regulatory_compliance) to perform thorough checks. "
            "Your final output must be a clear report with justifications."
        ),
        model="magistral-medium-latest", # As specified by you
        tools=[get_notams, check_regulatory_compliance], # Use the tools defined with @tool
        # Removed 'client' argument as it caused an error.
        # The Agent class likely handles client connection internally based on the model.
        # client=mistral_raw_client if 'mistral_raw_client' in locals() else None
    )
    print("RegulatoryRiskAgent created, using 'magistral-medium-latest' model via your agents framework.")

except Exception as e:
    print(f"An error occurred while creating the RegulatoryRiskAgent using agents.Agent: {e}")


# OpenAI Agent (GPT-4o for Route Optimization) - Using agents.Agent
# Defining the agent using the agents.Agent class to ensure compatibility with Runner/Handoff
try:
    route_optimizer_agent = Agent(
        name="RouteOptimizerAgent",
        instructions=(
            "You are an expert flight route optimizer. "
            "Your task is to propose the most efficient and safe flight path. "
            "Use your tools (get_weather_forecast, calculate_fuel_burn) to gather data. "
            "Your output must be a detailed flight route proposal with rationale."
        ),
        model="gpt-4o", # As specified by you
        tools=[get_weather_forecast, calculate_fuel_burn], # Use the tools defined with @tool
        # Removed 'client' argument as it caused an error.
        # The Agent class likely handles client connection internally based on the model.
        # client=openai_raw_client if 'openai_raw_client' in locals() else None
    )
    print("RouteOptimizerAgent created, using 'gpt-4o' model via your agents framework.")

except Exception as e:
    print(f"An error occurred while creating the OpenAI RouteOptimizerAgent: {e}")


# Instantiate Runner once, with no arguments.
workflow_runner = Runner()
print("Workflow Runner instantiated with no arguments.")

# Removing the explicit instantiation of handoff_task as handoff seems to be the callable itself
# handoff_task = handoff.Handoff()
# print("Handoff object instantiated.")


async def flight_planning_workflow(origin: str, destination: str, aircraft_type: str):
    print(f"\n--- Initiating Flight Planning for {origin} to {destination} ({aircraft_type}) ---")

    # --- Step 1: Route Optimization by OpenAI Agent (using Runner) ---
    print(f"\n[{route_optimizer_agent.name}] Optimizing route...")

    # Call Runner with the first agent and its input.
    # Based on previous errors, Runner.run likely takes the task/agent as the first argument
    # and the primary input as the second.
    optimization_request = (
        f"Propose the most efficient and safe flight route from {origin} to {destination} "
        f"for a {aircraft_type} aircraft. Consider current weather at both locations. "
        "Provide detailed reasoning for your chosen altitude, estimated time of arrival, and estimated fuel burn. "
        "Format your final output as: 'ROUTE PROPOSAL: [details] | REASONING: [explanation] | FUEL: [amount] | ETA: [time]' "
        "This precise format is crucial for the next agent."
    )

    try:
        # Running the agent defined using agents.Agent with the Runner
        initial_route_output = await workflow_runner.run(
            route_optimizer_agent,    # The agent (agents.Agent object)
            optimization_request,     # The input for the agent
        )
        print(f"\n[{route_optimizer_agent.name}] Raw Route Proposal Output:\n{initial_route_output.final_output}")

         # Extract just the route details for passing to the next agent
        route_details_part = ""
        try:
            # Adjusted splitting logic to be more robust
            output_parts = initial_route_output.final_output.split(" | ")
            route_proposal_part = next((part for part in output_parts if part.startswith("ROUTE PROPOSAL:")), None)
            if route_proposal_part:
                 route_details_part = route_proposal_part.replace("ROUTE PROPOSAL:", "").strip()
            else:
                 raise ValueError("Could not find 'ROUTE PROPOSAL:' in the output.")
        except (IndexError, ValueError) as e:
            print(f"Warning: Could not parse route details from {route_optimizer_agent.name}'s output. Using full output. Error: {e}")
            route_details_part = initial_route_output.final_output


    except Exception as e:
        initial_route_output_text = f"An error occurred during OpenAI Agent execution (via Runner): {e}"
        print(initial_route_output_text)
        # If the first agent fails, we might need to stop the workflow
        return initial_route_output_text



    # --- Step 2: Handoff to Regulatory & Risk Agent (Mistral) (using Runner and Handoff) ---
    print(f"\n[{regulatory_risk_agent.name}] Handoff initiated for compliance and risk assessment...")

    # Define the message for the regulatory risk agent
    regulatory_check_message = (
        f"The 'RouteOptimizerAgent' has proposed the following flight route. "
        f"Please meticulously review this route for full compliance with ICAO Annex 2 and national airspace regulations, "
        f"and assess all potential risks, considering current NOTAMs for {origin} and {destination}. "
        f"Provide a transparent, step-by-step justification for every finding. "
        f"Proposed Route Details: {route_details_part}. "
        f"Regulations to check: ICAO Annex 2, National Airspace Regulations. " # Explicitly listing for the agent
        f"Context for check: Origin={origin}, Destination={destination}."
    )

    try:
        # Use Runner.run to execute the handoff task.
        # Based on previous errors, 'handoff' itself is likely the callable task,
        # and Runner.run takes the handoff callable, target agent, and message as arguments.
        # The TypeError "takes 3 positional arguments but 4 were given" is still confusing.
        # Let's try passing them positionally again, as this matches the "takes 3 positional arguments" part of the error.
        final_report_output = await workflow_runner.run(
            handoff, # 1st positional argument
            regulatory_risk_agent, # 2nd positional argument
            regulatory_check_message # 3rd positional argument
        )

        print(f"\n--- Flight Planning Final Report (from {regulatory_risk_agent.name}) ---")
        print(final_report_output.final_output)
        return final_report_output.final_output

    except Exception as e:
        final_report_output_text = f"An error occurred during Handoff/Mistral Agent execution (via Runner): {e}"
        print(final_report_output_text)
        return final_report_output_text


# --- Main Execution ---
if __name__ == "__main__":
    # Need json for tool arguments parsing within native SDK flows if we were using them directly,
    # but the 'agents' library might handle this. Keeping the import just in case.
    import json
    current_time = datetime.now().strftime("%A, %B %d, %Y at %I:%M:%S %p %Z")
    print(f"\n--- Running workflow for current time: {current_time} from Montreal, Quebec, Canada ---\n")

    flight_origin = "Montreal, QC, Canada"
    flight_destination = "London, UK"
    flight_aircraft = "Boeing 787 Dreamliner"

    # Ensure mistral_raw_client and openai_raw_client are initialized before calling the workflow
    # This is handled in this cell before the __main__ block, so it should be fine.

    # Running the async workflow
    asyncio.run(flight_planning_workflow(flight_origin, flight_destination, flight_aircraft))

nest_asyncio applied to allow running asyncio.run in this environment.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
API keys loaded successfully.
Environment variables set.


RegulatoryRiskAgent created, using 'magistral-medium-latest' model via your agents framework.
RouteOptimizerAgent created, using 'gpt-4o' model via your agents framework.
Workflow Runner instantiated with no arguments.

--- Running workflow for current time: Wednesday, July 02, 2025 at 01:46:29 PM  from Montreal, Quebec, Canada ---


--- Initiating Flight Planning for Montreal, QC, Canada to London, UK (Boeing 787 Dreamliner) ---

[RouteOptimizerAgent] Optimizing route...
Creating trace Agent workflow with id trace_f9549f8069fb404eba3927eb410daf7f


DEBUG:openai.agents:Creating trace Agent workflow with id trace_f9549f8069fb404eba3927eb410daf7f


Setting current trace: trace_f9549f8069fb404eba3927eb410daf7f


DEBUG:openai.agents:Setting current trace: trace_f9549f8069fb404eba3927eb410daf7f


Creating span <agents.tracing.span_data.AgentSpanData object at 0x7d5ea1b74350> with id None


...

LLM resp:
[
  {
    "id": "msg_6865383d1a7c8195a09cb084f9debd870adaabea80bd8233",
    "content": [
      {
        "annotations": [],
        "text": "ROUTE PROPOSAL: Depart from Montreal-Trudeau International Airport, climb to an initial altitude of 35,000 feet, following the North Atlantic Track System (NATS) optimized routing to London Heathrow Airport. \n\nREASONING: \n1. **Weather Conditions**: Both departure and arrival destinations have clear skies and light winds, indicating stable weather and optimal flying conditions.\n2. **Altitude Selection**: The altitude of 35,000 feet is chosen to balance fuel efficiency with air traffic conditions typically seen at this time on transatlantic flights. This altitude also avoids any potential turbulence closer to the jet stream.\n3. **Routing via NATS**: Utilizing the North Atlantic Track System ensures that the route is as efficient as possible, relying on established pathways optimized for safety and fuel conservation. \n\nFUEL: 417.60 l

DEBUG:openai.agents:LLM resp:
[
  {
    "id": "msg_6865383d1a7c8195a09cb084f9debd870adaabea80bd8233",
    "content": [
      {
        "annotations": [],
        "text": "ROUTE PROPOSAL: Depart from Montreal-Trudeau International Airport, climb to an initial altitude of 35,000 feet, following the North Atlantic Track System (NATS) optimized routing to London Heathrow Airport. \n\nREASONING: \n1. **Weather Conditions**: Both departure and arrival destinations have clear skies and light winds, indicating stable weather and optimal flying conditions.\n2. **Altitude Selection**: The altitude of 35,000 feet is chosen to balance fuel efficiency with air traffic conditions typically seen at this time on transatlantic flights. This altitude also avoids any potential turbulence closer to the jet stream.\n3. **Routing via NATS**: Utilizing the North Atlantic Track System ensures that the route is as efficient as possible, relying on established pathways optimized for safety and fuel conservation

Resetting current trace


DEBUG:openai.agents:Resetting current trace



[RouteOptimizerAgent] Raw Route Proposal Output:
ROUTE PROPOSAL: Depart from Montreal-Trudeau International Airport, climb to an initial altitude of 35,000 feet, following the North Atlantic Track System (NATS) optimized routing to London Heathrow Airport. 

REASONING: 
1. **Weather Conditions**: Both departure and arrival destinations have clear skies and light winds, indicating stable weather and optimal flying conditions.
2. **Altitude Selection**: The altitude of 35,000 feet is chosen to balance fuel efficiency with air traffic conditions typically seen at this time on transatlantic flights. This altitude also avoids any potential turbulence closer to the jet stream.
3. **Routing via NATS**: Utilizing the North Atlantic Track System ensures that the route is as efficient as possible, relying on established pathways optimized for safety and fuel conservation. 

FUEL: 417.60 liters.

ETA: Approximately 6 hours and 30 minutes, considering current weather conditions and typical flight